In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from transformers import BitsAndBytesConfig, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

model_name = "Qwen/Qwen2.5-Math-7B-Instruct"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,                      # Загрузка модели в 4-битном формате
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,         # Использование вложенной квантизации для дополнительной экономии памяти
    bnb_4bit_quant_type="nf4",              # Метод квантизации: NormalFloat4
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
)
model.to('cuda')

tokenizer = AutoTokenizer.from_pretrained(model_name)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Загрузка датасета
ds = load_dataset("ScalableMath/Lean-STaR-plus")
ds = ds['train'].train_test_split(test_size=0.01, seed=42)
ds_train = ds['train']
ds_test = ds['test']

def format_instruction(example):
    return {
        "text": f"{example['input']}\n{example['output']}"
    }

formatted_ds = ds_train.map(format_instruction)

def tokenize_function(examples):
    result = tokenizer(
        examples["text"],
        truncation=True,
        max_length=2048,
        padding="max_length",
    )
    result["labels"] = result["input_ids"].copy()
    return result

tokenized_ds = formatted_ds.map(
    tokenize_function,
    batched=True,
    remove_columns=["text"],
    desc="Tokenizing dataset",
)

model = prepare_model_for_kbit_training(model)

# Определение конфигурации LoRA
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_dropout=0,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)
print(model.print_trainable_parameters())

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, 
    mlm=False,
)

# Настройка параметров обучения
training_args = TrainingArguments(
    output_dir="./qwen-math-qlora",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    save_steps=1000,
    save_total_limit=20,
    learning_rate=2e-4,
    optim="paged_adamw_8bit",
    weight_decay=0.01,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    logging_steps=500,
    logging_dir="./logs",
    bf16=True,
    gradient_checkpointing=True,
    seed=42,
)

# Инициализация Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds,
    data_collator=data_collator,
)

# Обучение модели
trainer.train()

# Сохранение адаптера и токенизатора
model.save_pretrained("./qwen-math-qlora")
tokenizer.save_pretrained("./qwen-math-qlora")

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


trainable params: 40,370,176 || all params: 7,655,986,688 || trainable%: 0.5273
None


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
500,0.890200
1000,0.512200


In [1]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Загрузка конфигурации адаптера
config = PeftConfig.from_pretrained("./qwen-math-qlora/checkpoint-1000")

# Загрузка базовой модели
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

# Применение QLoRA адаптера
model = PeftModel.from_pretrained(model, "./qwen-math-qlora/checkpoint-1000")

# Загрузка токенизатора
tokenizer = AutoTokenizer.from_pretrained("./qwen-math-qlora/checkpoint-1000")

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [7]:
# Пример генерации
input_text = """<|im_start|>user
My LEAN 4 state is:
import Mathlib.Data.Matrix.Basic
import Aesop
import Mathlib.Tactic.NormNum
import Mathlib.Tactic.RewriteSearch
import AutoSolver


@[simp] def x: List ℚ :=  [6, 2, 9]
@[simp] def e_1: List ℚ :=  [5, 0, 4]
@[simp] def e_2: List ℚ :=  [5, -1, 0]
@[simp] def e_3: List ℚ :=  [-1, 0, 4]

@[simp] def x1: ℚ := 73/24
@[simp] def x2: ℚ := -2
@[simp] def x3: ℚ := -19/24
Please write down the reasoning that leads to the possible next tactic and then predict the tactic to help me prove the corectness of the system.<|im_end|> \
<|im_start|>assistant"""
input_ids = tokenizer(input_text, return_tensors="pt").to(model.device)
outputs = model.generate(**input_ids, max_length=512, temperature=0.7)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

user
My LEAN 4 state is:
import Mathlib.Data.Matrix.Basic
import Aesop
import Mathlib.Tactic.NormNum
import Mathlib.Tactic.RewriteSearch
import AutoSolver


@[simp] def x: List ℚ :=  [6, 2, 9]
@[simp] def e_1: List ℚ :=  [5, 0, 4]
@[simp] def e_2: List ℚ :=  [5, -1, 0]
@[simp] def e_3: List ℚ :=  [-1, 0, 4]

@[simp] def x1: ℚ := 73/24
@[simp] def x2: ℚ := -2
@[simp] def x3: ℚ := -19/24
Please write down the reasoning that leads to the possible next tactic and then predict the tactic to help me prove the corectness of the system. assistant

Here is the reasoning:
To prove the goal, we need to show that the given list of rational numbers is a basis for the vector space. This can be achieved by using the fact that the list of rational numbers is linearly independent and spans the vector space.
Here is the predicted next tactic:
```lean
exact Aesop.basis (Aesop.quotient 3) (Aesop.quotient 3) x e_1 e_2 e_3
```
